In [68]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import gradio as gr

In [69]:
data=pd.read_excel('.\paragraphs.xlsx')
x=data.columns[2]
paragraphs=list(data[x])
paragraphs.append(x)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\hardi\AppData\Local\Temp\ipykernel_9804\2665997817.py:1: SyntaxWarning: invalid escape sequence '\p'
  data=pd.read_excel('.\paragraphs.xlsx')


In [70]:
queries=pd.read_excel('.\queries.xlsx')
queries.loc[len(queries.index)]=queries.columns
queries.head()

<>:1: SyntaxWarning: invalid escape sequence '\q'
<>:1: SyntaxWarning: invalid escape sequence '\q'
C:\Users\hardi\AppData\Local\Temp\ipykernel_9804\529053342.py:1: SyntaxWarning: invalid escape sequence '\q'
  queries=pd.read_excel('.\queries.xlsx')


,What is the parts of the trophy?,True,"['- the cup itself, plus a lid and a base']"
0,"If you need a date of four days less, what do ...",True,['-4']
1,What was Hanover's attitude toward Austria?,True,['-belligerent Hanover']
2,What is an example of a suffix used by Old Eng...,True,['-de']
3,The Modern English ending -'s is derived from ...,True,['-es']
4,How are the games broadcast now?,True,"[', FA Cup matches are shown live by ITV acros..."


In [71]:
queries.columns=['query','possiblity','Result']
queries.head()

,query,possiblity,Result
0,"If you need a date of four days less, what do ...",True,['-4']
1,What was Hanover's attitude toward Austria?,True,['-belligerent Hanover']
2,What is an example of a suffix used by Old Eng...,True,['-de']
3,The Modern English ending -'s is derived from ...,True,['-es']
4,How are the games broadcast now?,True,"[', FA Cup matches are shown live by ITV acros..."


In [72]:
#Filling Null Values with "" in data
paragraphs = [str(paragraph) if paragraph is not np.nan else "" for paragraph in paragraphs]
queries['query'] = queries['query'].fillna("")


In [73]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(paragraphs)#Tokenizing Paragraphs

In [74]:
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\hardi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [75]:
def similar_paras(query):
    query_ = vectorizer.transform([query])#converting the query to vector form
    similarity = cosine_similarity(query_, X).flatten()#calculating similarity b/w query vector and the paragraphs vector
    indexes = similarity.argsort()[-5:][::-1] #Extracting indexes of the 5 most similar paragraphs to the query
    return [(paragraphs[i], similarity[i]) for i in indexes]

In [76]:
def get_answer(query, paragraph):
    result = qa_pipeline(question=query, context=paragraph)
    return result['answer'], result['score']


In [77]:
def ans_from_query(query):
    similar_paragraphs = similar_paras(query)
    answer_found = False
    # answer="None"
    for para, similarity in similar_paragraphs:
        answer, score = get_answer(query, para)
        if score > 0.5:
            print(answer)
            return {'query': query, 'paragraph': para, 'answer': answer}
    return {'query': query, 'paragraph': None, 'answer':'NONE'}

In [80]:
# # print(ans_from_query("How are the games broadcast now?"))

# for query in queries['query']:
#     result=ans_from_query(query)

In [79]:
def predict_gen(Query):
    Result=ans_from_query(Query)
    return Result['answer']
interface= gr.Interface(fn=predict_gen,inputs="text",outputs="text")
interface.launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
